<a href="https://colab.research.google.com/github/Sergio-Per/MY-GUI-experiments/blob/main/MY_FINAL_PROJECT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import scipy as sp
%matplotlib inline


from ipywidgets import GridspecLayout
import ipywidgets as widgets

from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from pandas.plotting import autocorrelation_plot,lag_plot

from statsmodels.tsa.stattools import adfuller,kpss#,range_unit_root_test,zivot_andrews

from scipy.fftpack import fft,fftfreq,ifft

import datetime

from google.colab import files
import io

from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

import matplotlib.patches as patches

import statsmodels.api as sm


/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [3]:
uploaded = files.upload()

Saving black_carbon.csv to black_carbon.csv


In [4]:
bc = pd.read_csv(io.BytesIO(uploaded['black_carbon.csv']), na_values=('--'), parse_dates=["Datetime"])

bc= bc.set_index('Datetime')


bc['Year'] = pd.DatetimeIndex(bc.index).year
bc['Month'] = pd.DatetimeIndex(bc.index).month
bc['Day'] = pd.DatetimeIndex(bc.index).day
bc['Hour'] = pd.DatetimeIndex(bc.index).hour
bc['Weekday'] = pd.DatetimeIndex(bc.index).weekday

bc["BC"].fillna(bc["BC"].mean(), inplace=True)

bc.BC[bc['BC']<0.01] = 0.01
bc.head()

,T_amb,wind_speed,wind_dir,sc550,AE_450_700,PM10,BC,Year,Month,Day,Hour,Weekday
Datetime,,,,,,,,,,,,
2007-04-24 12:00:00,23.98,1.63,202.81,87.62,1.42935,63.89272,2.47,2007.0,4.0,24.0,12.0,1.0
2007-04-24 13:00:00,24.99,1.73,201.25,77.56,1.46802,63.51981,2.29,2007.0,4.0,24.0,13.0,1.0
2007-04-24 14:00:00,24.73,1.60,214.79,78.38,1.54528,59.83932,2.84,2007.0,4.0,24.0,14.0,1.0
2007-04-24 15:00:00,24.59,1.25,205.39,75.74,1.56389,61.13641,2.45,2007.0,4.0,24.0,15.0,1.0
2007-04-24 16:00:00,25.82,1.49,209.38,77.56,1.55272,66.60039,2.46,2007.0,4.0,24.0,16.0,1.0


In [5]:
hourly = bc[(bc.index < '2011-04-24 12:00:00')].copy()



daily = hourly.resample("D").mean()
daily_sd = hourly.resample("D").std()


weekly = hourly.resample("W").mean()
weekly_sd = hourly.resample("W").std()

monthly = hourly.resample("M").mean()
monthly_sd = hourly.resample("M").std()

quarterly = hourly.resample("Q").mean()
quarterly_sd = hourly.resample("Q").std()


yearly =hourly.resample("Y").mean()
yearly_sd = hourly.resample("Y").std()


samples_list = [hourly, daily, weekly, monthly, quarterly, yearly]


# **Descritive Statistics**

In [6]:
import warnings
warnings.filterwarnings('ignore')

ALL = 'ALL'
def unique_sorted_values_plus_ALL(array):
  unique=array.unique().tolist()
  unique.sort()
  unique.insert(0,ALL)
  return unique




output = widgets.Output()
plot_output =widgets.Output()

dropdown_year = widgets.Dropdown(options = unique_sorted_values_plus_ALL(hourly.Year), description='YEAR')
dropdown_month = widgets.Dropdown(options = unique_sorted_values_plus_ALL(hourly.Month), description='MONTH')
dropdown_day = widgets.Dropdown(options = unique_sorted_values_plus_ALL(hourly.Day), description='DAY')


def common_filtering(Year, Month, Day):



   

  output.clear_output()

  plot_output.clear_output()

  
  
  if (Year== ALL) & (Month==ALL) & (Day==ALL):
    common_filter = hourly
  
  elif (Year== ALL) & (Month==ALL):
    common_filter = hourly[hourly.Day == Day]

  elif (Year== ALL) & (Day==ALL):
    common_filter = hourly[hourly.Month == Month]

  elif (Month== ALL) & (Day==ALL):
    common_filter = hourly[hourly.Year == Year]       

  elif (Year== ALL):
    common_filter = hourly[(hourly.Month == Month) & (hourly.Day == Day)]

  elif (Month== ALL):
    common_filter = hourly[(hourly.Year == Year) & (hourly.Day == Day)]

  elif (Day==ALL):
    common_filter = hourly[(hourly.Year == Year) & (hourly.Month == Month)]
    
  else:
    common_filter = hourly[(hourly.Year == Year) & (hourly.Month == Month) & (hourly.Day == Day)]


  with output:
    display(common_filter)
  
  with plot_output:

    
    fig = plt.figure(figsize=(15,10))

    fig.patch.set_facecolor('#E0E0E0')
    fig.patch.set_alpha(0.7)

    plt.suptitle('Time-Series Descriptive Statistics',fontsize=16, alpha=0.9, color = 'black')
    ax1 = plt.subplot(2,3,1)

    ax2 = plt.subplot(2,3,2)

    ax3 = plt.subplot(2,3,3)

    ax4 = plt.subplot(2,1,2)

    axes = [ax1, ax2, ax3, ax4]
    for ax in axes:
      ax.set_facecolor('whitesmoke')

    # fig,(ax1, ax2, ax3,ax4)=plt.subplots(1,4, figsize=(25,5), gridspec_kw={'width_ratios': [1.5, 1, 4, 2]})
    # fig.suptitle('Time-Series Descriptive Statistics',fontsize=16, alpha=0.9, color = 'black')
    # fig.set_facecolor('gainsboro')

    # for ax in [ax1,ax2,ax3,ax4]:
    #   ax.set_facecolor('whitesmoke')

    ax1.set_title('Year/Month/Day :  '+ str(Year) + '/' + str(Month) + '/' + str(Day))
    ax1.grid(True, linestyle='-.', linewidth = 0.5, alpha = 1, color='grey')
    sns.set_context("notebook", font_scale=1, rc={"lines.linewidth": 1.5})
    # ax1.text(3.4, 0.1, 'year : '+str(Year))
    # ax1.text(3.4, 0.16, 'month : '+str(Month))
    # ax1.text(3.4, 0.22, 'day : '+str(Day))
    #ax1.axvline(str(round(common_filter['BC'].mean(),2)))
    #ax1.axvline(str(round(common_filter['BC'].median(),2)))
    
    # ax.text(0.5, 2.5, 'insert text in plot')
    with sns.axes_style("whitegrid", {"axes.facecolor": "1"}):
      sns.kdeplot(common_filter['BC'], color='c',shade=False, ax=ax1)
      sns.distplot(common_filter['BC'], ax=ax1)
      ax1.set_xlim(-1,common_filter['BC'].mean()+5)
     # ax1.set_ylim(0)

    #ax2.plot(common_filter['BC'])

    column_labels=["Column 1"]
    df=pd.DataFrame([round(common_filter['BC'].count(),2),
                     round(common_filter['BC'].mean(),2),
                     round(common_filter['BC'].std(),2),
                     round(common_filter['BC'].min(),2),
                     round(common_filter['BC'].quantile(0.01),2),
                     round(common_filter['BC'].quantile(0.05),2),
                     round(common_filter['BC'].quantile(0.25),2),
                     round(common_filter['BC'].median(),2),
                     round(common_filter['BC'].quantile(0.75),2),
                     round(common_filter['BC'].quantile(0.95),2),
                     round(common_filter['BC'].quantile(0.99),2),
                     round(common_filter['BC'].max(),2),
                     round(common_filter['BC'].quantile(0.75)-common_filter['BC'].quantile(0.25),2),
                     round(common_filter['BC'].skew(),2), round(common_filter['BC'].kurtosis(),2)],columns=column_labels)
    ax2.axis('tight')
    
    #ax1.set_title('Values distribution')
    
    ax2.axis('off')
    ax2.table(cellText=df.values, colLabels=['BC Stats'], 
              rowLabels=['# Data',"Mean","Std",'Min','P1','P5' ,'P25','Median','P75','P95', 'P99','Max', 'IQR', 'Skew', 'Kurt'], 
              rowColours =["lightgrey"]*len(df) , colColours =["lightgrey"], loc='center',bbox = (0.35,0,.3,1))
    
  

    
    ax3.grid(True, linestyle='-.', linewidth = 0.5, alpha = 1, color='grey')
    # ax3=plt.gca()
    # ax3.set_aspect(0.01)
    sns.boxenplot(y=common_filter['BC'], color='royalblue', ax=ax3)
    

    ax4.plot(common_filter['BC'], c='black')
    ax4.fill_between(common_filter.index,common_filter['BC'].quantile(0.25), common_filter['BC'].quantile(0.75),alpha=0.4)
    ax4.fill_between(common_filter.index,common_filter['BC'].mean()-common_filter['BC'].std(), common_filter['BC'].mean()+common_filter['BC'].std(),alpha=0.15)

    ax4.axhline(common_filter['BC'].mean(), alpha=0.7)
    #ax4.set_ylim(0,common_filter['BC'].max())
    plt.show()
  



#sns.JointGrid(data=common_filter, x="Weekday", y="BC",ax=ax3)
def dropdown_year_eventhandler(change):
  common_filtering(change.new, dropdown_month.value,dropdown_day.value)

def dropdown_month_eventhandler(change):
  common_filtering(dropdown_year.value, change.new,dropdown_day.value )  

def dropdown_day_eventhandler(change):
  common_filtering(dropdown_year.value, dropdown_month.value, change.new)

dropdown_year.observe(dropdown_year_eventhandler, names='value')
dropdown_month.observe(dropdown_month_eventhandler, names='value')
dropdown_day.observe(dropdown_day_eventhandler, names='value')




# display(dropdown_year)
# display(dropdown_month)

# display(output)
# display(plot_output)


input_widgets = widgets.HBox([dropdown_year, dropdown_month, dropdown_day])
#display(input_widgets)


tab = widgets.Tab([output, plot_output])
tab.set_title(0, 'DATASET')
tab.set_title(1, 'STATS')
tab.set_title(2, 'TIMESERIES')
# tab.set_title(2, 'TIMESERIES')
#display(tab)


dashboard = widgets.VBox([input_widgets, tab])
display(dashboard)

The user is able to choose between all data or a particular day, month and year, or all day 

# **Resampling, difference, moving average, autocorrelation, stationarity**

In [7]:
def my_figure1(n_days=14, yi=2007, yf=2012, lags = 50, dt_diff=1):


    ts1 = daily.BC[str(yi):str(yf)]
    ts2 = daily.BC[str(yi):str(yf)].resample(str(n_days)+'D').mean()
    ts3 = daily.BC[str(yi):str(yf)].resample(str(n_days)+'D').std()
    ts4 = daily.BC[str(yi):str(yf)].resample(str(n_days)+'D').median()
    
    ts_diff = ts2 - ts2.shift(dt_diff)
    ts_diff.dropna(inplace = True)
    
#     print(ts2.head())
#     print(ts2.tail())
#------------ Results of kpss test and result----------------------------      
    
    k_test =  kpss(ts2, regression ='ct')
   
    koutput = pd.Series(k_test[0:2], index = ['Test Statistic','p-value'])
   
    for key,value in k_test[3].items():
        koutput['Critical Value (%s)'%key] = value
#     print('-------------------'*3)    
#     print('Results of KPSS test:')
#     print(koutput)
#     print('-------------------'*3)

    
#------------ Results of adfuller test and result----------------------------  


    af = adfuller(ts2, regression = 'ctt', autolag = 'AIC')
    
    dfoutput = pd.Series(af[0:4], index = ['Test Statistic','p-value','#Lags Used','Number of Observations Used'])
   
    for key,value in af[4].items():
        dfoutput['Critical Value (%s)'%key] = value
        
    # print('-------------------'*3)    
    # print('Results of Dickey-Fuller Test:')
    # print(dfoutput)
    # print('-------------------'*3)

#------------ Plotting time series----------------------------


    fig = plt.figure(figsize=(16,10))

    fig.patch.set_facecolor('#E0E0E0')
    fig.patch.set_alpha(0.7)

    ax1 = plt.subplot(2,3,1)
    ax2 = plt.subplot(2,3,2)
    ax3 = plt.subplot(2,3,3)
    ax4 = plt.subplot(2,1,2)
    axes = [ax1, ax2, ax3, ax4]


  #  fig,((ax1,ax2),(ax3,ax4)) = plt.subplots(2,2,figsize=(15,7))
    
    ax1.set_title('Time series \n Hourly BC measurements')
    ax1.set_facecolor('whitesmoke')
    
    
    
#    ax.scatter(ts1.index,ts1,c='black',alpha=0.1, label = 'hourly values, initial data')
    ax4.plot(ts1,c='black',alpha=0.22, label = 'hourly values, initial data')
    ax4.plot(ts2, c='darkblue',linewidth = 2, label = 'resampled values, mean') 
    ax4.plot(ts4, c='blue',linewidth = 2, alpha =1, label = 'resampled values, median')
    
    ax4.fill_between(ts2.index,ts2-ts3, ts2+ts3, alpha = 0.5,  label = 'std')
    ax4.plot(ts2+ts3, c = 'darkblue', linewidth = .5, alpha=1)
    ax4.plot(ts2-ts3, c = 'darkblue', linewidth = .5, alpha=1)
    
    ax4.set_ylim(0,(ts2+ts3).max()+2)    
    ax4.legend( loc='best', shadow=True, fontsize='medium')

    
    ax1.plot(ts_diff, c='darkblue',linewidth = 2, label = 'diff')
    ax1.set_title('Difference ts for selected \n resampling period, time span and lag')
    ax1.set_facecolor('whitesmoke')
    ax1.legend( loc='best', shadow=True, fontsize='medium')

    ax2.set_title('Autocorrelation for selected \n resampling period and time span')
    ax2.set_facecolor('whitesmoke')
    autocorrelation_plot(ts2,ax=ax2)
#     plot_acf(ts2, lags = lags ,label = 'acf',ax = ax2)
#     plot_pacf(ts2, lags = lags ,method='ywm',label = 'pacf',ax = ax2)
#    ax2.legend( loc='best', shadow=True, fontsize='medium')
    ax2.set_xlim(0,lags) 
    
    
    ax3.set_title('Autocorrelation diff for selected: \n -resampling period, time span and lag')
    ax3.set_facecolor('whitesmoke')
    autocorrelation_plot(ts_diff,ax=ax3)
#     plot_acf(ts_diff, lags = lags ,label = 'acf', c = 'green',ax = ax4)
#     plot_pacf(ts_diff, lags = lags ,method='ywm',label = 'pacf',ax = ax4)
#    ax4.legend( loc='best', shadow=True, fontsize='medium')
    ax3.set_xlim(0,lags)
    
    
n_days = widgets.IntSlider(value=14,min=2,max=180,step=2,description='n days : ',disabled=False,continuous_update=False,
                           orientation='horizontal', readout=True, readout_format='d')

lags = widgets.IntSlider(value=50,min=2,max=100,step=1,description='Lag : ',disabled=False,continuous_update=False,
                           orientation='horizontal', readout=True, readout_format='d')

dt_diff = widgets.IntSlider(value=1,min=1,max=60,step=1,description='day diff : ',disabled=False,continuous_update=False,
                           orientation='horizontal', readout=True, readout_format='d')


interact(my_figure1 , n_days=n_days, yi=(2007,2012), yf=(2007,2012), lags = lags, dt_diff = dt_diff);

interactive(children=(IntSlider(value=14, continuous_update=False, description='n days : ', max=180, min=2, st…

# **Spectral Analysis, FFT,** 

In [10]:
hourly = bc[(bc.index < '2011-04-24 12:00:00')].copy()
import warnings
warnings.filterwarnings('ignore')

def my_figure_FFT( n_hours = 1):

  t = np.arange(len(np.array(hourly.BC[:'2011'])))
  ts_resampled = hourly.BC[:'2011'].rolling(n_hours, min_periods=1).mean()

  #ts_resampled = hourly.BC[:'2011'].resample(str(n_hours)+'H').mean()



  sig_fft_original = fft(np.array(hourly.BC[:'2011']))
  sig_fft = fft(np.array(ts_resampled))
  sig_fft_filters = fft(np.array(hourly.BC[:'2011']))
    # copy the FFT results
  
    # obtain the frequencies using scipy function
  freq_original = fftfreq(len(hourly.BC[:'2011']), d=1)
  freq = fftfreq(len(np.array(ts_resampled)), d=1)

    # copy the FFT results
  sig_fft_filtered_low = sig_fft_filters.copy()
  sig_fft_filtered_high = sig_fft_filters.copy()
    # obtain the frequencies using scipy function
  freq_low = fftfreq(len(np.array(hourly.BC[:'2011'])), d=1)
  freq_high = fftfreq(len(np.array(hourly.BC[:'2011'])), d=1)
    # define the cut-off frequency
    #cut_off = 0.0006

    # low-pass filter 

  # sig_fft_filtered_low[(np.abs(freq_low)) > (cut_off_l)] = 0
    
  # sig_fft_filtered_high[(np.abs(freq_high)) < (cut_off_h)] = 0

    # get the filtered signal in time domain
  filtered_low = ifft(sig_fft_filtered_low)
  filtered_high = ifft(sig_fft_filtered_high)





  fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15,4),gridspec_kw={'width_ratios': [3, 1.5]})
  fig.set_facecolor('#E0E0E0')
  ax1.set_title('n hours = ' + str(n_hours))
  ax1.plot(t,hourly.BC[:'2011'], c='black', alpha = 0.1)
  ax1.plot(t, ts_resampled, c='black', alpha = 1)
   
  ax2.plot(1/freq_original, np.abs(sig_fft_original),c='black')
  ax2.plot(1/freq, np.abs(sig_fft))
  ax2.set_title('Amplitude Spectrum')
  ax2.set_xticks([12, 24, 48, 72, 84, 168])
  #ax2.set_xlim(0, 200)
  #ax2.set_ylim(10, 10000)
  ax2.set_xlabel('Hour')
  ax2.set_ylabel('FFT Amplitude');
  ax2.set_yscale("log")
  ax2.set_xscale("log");
  plt.show()

   
    
# cut_off_l = widgets.FloatLogSlider(value=1e-3,base=10,min=-5, max=1, step=0.01, description='Low Pass cut off', 
#                                    orientation='horizontal', readout_format='.5f')

# cut_off_h = widgets.FloatLogSlider(value=0.004,base=10,min=-6, max=-0.30, step=0.01, description='High Pass cut off', 
#                                    orientation='horizontal', readout_format='.5f')

n_hours = widgets.IntSlider(value=1,min=1,max=10000,step=1,description='rolling_hours : ',disabled=False,continuous_update=False,
                            orientation='horizontal', readout=True, readout_format='d')


interact(my_figure_FFT , n_hours=n_hours )

#################################################################################
####################################################################################


def my_figure_Filters(cut_off_l = 7e-3,cut_off_h = 0.004):



#plot the filtered signal


  t = np.arange(len(np.array(hourly.BC[:'2011'])))

  sig_fft_original = fft(np.array(hourly.BC[:'2011']))
  
  sig_fft_filters = fft(np.array(hourly.BC[:'2011']))
    # copy the FFT results
  
    # obtain the frequencies using scipy function
  freq_original = fftfreq(len(hourly.BC[:'2011']), d=1)
  

    # copy the FFT results
  sig_fft_filtered_low = sig_fft_filters.copy()
  sig_fft_filtered_high = sig_fft_filters.copy()
    # obtain the frequencies using scipy function
  freq_low = fftfreq(len(np.array(hourly.BC[:'2011'])), d=1)
  freq_high = fftfreq(len(np.array(hourly.BC[:'2011'])), d=1)
    # define the cut-off frequency
    #cut_off = 0.0006

    # low-pass filter 

  sig_fft_filtered_low[(np.abs(freq_low)) > (cut_off_l)] = 0
    
  sig_fft_filtered_high[(np.abs(freq_high)) < (cut_off_h)] = 0

    # get the filtered signal in time domain
  filtered_low = ifft(sig_fft_filtered_low)
  filtered_high = ifft(sig_fft_filtered_high)



  fig, (ax_low,ax_high) = plt.subplots(2,1, figsize = (12, 5))
  fig.set_facecolor('#E0E0E0') 
  ax_low.set_title('Low Pass Filter on the signal, cutoff = ' + str(round(365*cut_off_l,6))+ '(day)')
  ax_low.plot(t,hourly.BC[:'2011'], c='black', alpha = 0.1)
  ax_low.plot(t, filtered_low, c = 'blue',linewidth = 2.5)
  ax_low.set_xlim(-500,35000)
  ax_low.set_ylim(0, filtered_low.max()+1)
    
  ax_low.set_xlabel('Time (h) ')
  ax_low.set_ylabel('Amplitude')

  ax_high.set_title('High Pass Filter on the signal, cutoff = ' + str(round(365*cut_off_h,6))+ '(day)')
  ax_high.plot(t,hourly.BC[:'2011'], c='black', alpha = 0.1)
  ax_high.plot(t, filtered_high, c = 'blue',linewidth = 2.5)
  ax_high.set_xlim(-500,35000)
  #ax_high.set_ylim(0, filtered_high.max()+1)
  # ax_high.set_yscale("log")  
  ax_high.set_xlabel('Time (h) ')
  ax_high.set_ylabel('Amplitude')


cut_off_l = widgets.FloatLogSlider(value=1e-3,base=10,min=-5, max=1, step=0.01, description='Low Pass cut off', 
                                   orientation='horizontal', readout_format='.5f')

cut_off_h = widgets.FloatLogSlider(value=0.004,base=10,min=-6, max=-0.30, step=0.01, description='High Pass cut off', 
                                   orientation='horizontal', readout_format='.5f')

interact(my_figure_Filters ,cut_off_l=cut_off_l,cut_off_h=cut_off_h)




interactive(children=(IntSlider(value=1, continuous_update=False, description='rolling_hours : ', max=10000, m…

interactive(children=(FloatLogSlider(value=0.001, description='Low Pass cut off', max=1.0, min=-5.0, readout_f…

<function __main__.my_figure_Filters>

## SARIMAX-Seasonal AutoRegressive Integrated Moving Average with eXogenous regressors model